In [1]:
import __init__

from collections import defaultdict
from functools import partial
from typing import Callable

import numpy as np
from cv2.typing import MatLike
from eval import cv2_to_pil, load_data, pil_to_cv2
from eval_with_ground import evaluate_image_pair_from_pil
from eval_without_ground import evaluate_image
from tqdm import tqdm

from dehaze import dehaze_and_enhance

In [2]:
paired_samples = load_data()

100%|██████████| 7200/7200 [01:16<00:00, 93.64it/s] 


In [7]:
from PIL import Image
for k in paired_samples.keys():
    p = paired_samples[k][0]
    low_image = p['low_image']
    high_image = p.get('high_image', None)
    name = p['name']
    low_image.save(f'./show/{name}_low.png')
    if high_image is not None:
        high_image.save(f'./show/{name}_high.png')

In [4]:
def eval(dataset, method: Callable[[MatLike], MatLike]):
    samples = paired_samples[dataset]
    results = defaultdict(list)
    for sample in tqdm(samples):
        low_image = sample["low_image"]
        high_image = sample.get("high_image")
        # print(low_image, high_image)
        if low_image is not None:
            reference_img = cv2_to_pil(method(pil_to_cv2(low_image)))
            # reference_img.show()
            if high_image is not None:
                scores = evaluate_image_pair_from_pil(reference_img, high_image)
                # Compute metrics here
            else:
                scores = evaluate_image(reference_img, low_image)
            for key, s in scores.items():
                results[key].append(s)
    return results

In [5]:
paired_samples.keys()

dict_keys(['lol_dataset', 'Dark_Face', 'LOL-v2'])

# dehaze

In [5]:
result = eval('lol_dataset', dehaze_and_enhance)

100%|██████████| 500/500 [00:19<00:00, 26.07it/s]


In [6]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 3876.0293 ± 3862.0581
PSNR: 14.2037 ± 4.2630
SSIM: 0.5057 ± 0.1516


In [6]:
result = eval('Dark_Face', dehaze_and_enhance)

100%|██████████| 501/501 [00:36<00:00, 13.62it/s]


In [7]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

SIMPLE_SCORE: 60.5843 ± 20.0145
CEI: 1.9076 ± 0.4288


In [9]:
result = eval('LOL-v2', dehaze_and_enhance)

100%|██████████| 100/100 [00:03<00:00, 30.36it/s]


In [10]:
for key, r in result.items():
    print(f"{key}: {np.mean(r):.4f} ± {np.std(r):.4f}")

MSE: 2107.1069 ± 1780.1074
PSNR: 16.2354 ± 3.3389
SSIM: 0.5460 ± 0.1234
